<a href="https://colab.research.google.com/github/silassjjunior/Sistema-de-Recomenda-o-Atrav-s-de-Imagens/blob/main/Recomenda%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

#  INSTALAÇÃO E IMPORTAÇÃO DAS BIBLIOTECAS
!pip install ipywidgets joblib

import os
import joblib
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from IPython.display import display, clear_output
import logging

import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model

import ipywidgets as widgets
from sklearn.metrics.pairwise import cosine_similarity

print(">>> Bibliotecas importadas com sucesso!")

# Habilita widgets
!jupyter nbextension enable --py widgetsnbextension

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")


# CLASSE SISTEMA DE RECOMENDAÇÃO

class SistemaRecomendacao:
    def __init__(self, diretorio_dataset="dataset", tamanho_alvo=(224, 224), batch_size=32):
        self.diretorio_dataset = diretorio_dataset
        self.tamanho_alvo = tamanho_alvo
        self.batch_size = batch_size
        self.modelo_extrator = self._carregar_modelo_extrator()
        self.vetores_caracteristicas = []
        self.caminhos_imagens = []
        self.caminho_busca_atual = None

        # Pastas
        os.makedirs(self.diretorio_dataset, exist_ok=True)
        os.makedirs("uploads", exist_ok=True)
        os.makedirs("dataset_padronizado", exist_ok=True)

        # Widgets
        self._criar_widgets()

    def _criar_widgets(self):
        self.uploader = widgets.FileUpload(
            accept='image/*',
            description='Carregar Imagem',
            button_style='primary'
        )
        self.btn_treinar = widgets.Button(description="Executar Treinamento", button_style='danger', icon='cogs')
        self.output_area = widgets.Output()
        self.progress_bar = widgets.FloatProgress(value=0.0, min=0.0, max=1.0, description='Progresso:')

        # Callbacks
        self.uploader.observe(self._handle_upload, names='value')
        self.btn_treinar.on_click(self._on_click_treinar)

    def _carregar_modelo_extrator(self):
        try:
            base_model = VGG16(weights='imagenet', include_top=False, input_shape=(*self.tamanho_alvo, 3))
            model = Model(inputs=base_model.input, outputs=base_model.output)
            logging.info("Modelo VGG16 carregado como extrator de características.")
            return model
        except Exception as e:
            logging.error(f"Erro ao carregar o modelo VGG16: {e}")
            with self.output_area:
                print(f"Erro ao carregar o modelo VGG16: {e}")
            return None

    def _extrair_caracteristicas_imagem(self, caminho_imagem):
        try:
            img = image.load_img(caminho_imagem, target_size=self.tamanho_alvo)
            img_array = image.img_to_array(img)
            preprocessed_img = preprocess_input(np.expand_dims(img_array, axis=0))
            features = self.modelo_extrator.predict(preprocessed_img, verbose=0)
            normalized = features.flatten() / np.linalg.norm(features.flatten())
            return normalized
        except Exception as e:
            logging.error(f"Erro ao extrair características de {caminho_imagem}: {e}")
            return None

    def carregar_dados_treinados(self):
        try:
            self.vetores_caracteristicas = joblib.load("vetores_caracteristicas.joblib")
            self.caminhos_imagens = joblib.load("caminhos_imagens.joblib")
            if not self.vetores_caracteristicas:
                return False
            with self.output_area:
                print(f"Dados carregados. {len(self.vetores_caracteristicas)} produtos no catálogo.")
            return True
        except:
            return False

    def treinar(self):
        with self.output_area:
            clear_output()
            print("--- INICIANDO TREINAMENTO ---")
        arquivos_no_dir = [f for f in os.listdir(self.diretorio_dataset) if os.path.isfile(os.path.join(self.diretorio_dataset, f))]
        if not arquivos_no_dir:
            with self.output_area:
                print("Nenhuma imagem encontrada no dataset.")
            return

        caminhos_validos = []
        self.progress_bar.value = 0
        self.progress_bar.description = 'Padronizando:'

        for i, filename in enumerate(arquivos_no_dir):
            caminho_origem = os.path.join(self.diretorio_dataset, filename)
            caminho_destino = os.path.join("dataset_padronizado", filename)
            try:
                with Image.open(caminho_origem) as img:
                    img_rgb = img.convert('RGB')
                    if img_rgb.size != self.tamanho_alvo:
                        img_rgb = img_rgb.resize(self.tamanho_alvo, Image.LANCZOS)
                    img_rgb.save(caminho_destino)
                    caminhos_validos.append(caminho_destino)
            except:
                pass
            self.progress_bar.value = (i + 1) / len(arquivos_no_dir)

        self.caminhos_imagens = caminhos_validos
        self.progress_bar.value = 0
        self.progress_bar.description = 'Extraindo:'

        lista_caracteristicas = []
        for i in range(0, len(caminhos_validos), self.batch_size):
            batch_paths = caminhos_validos[i:i+self.batch_size]
            batch_imgs = []
            for p in batch_paths:
                try:
                    img = image.load_img(p, target_size=self.tamanho_alvo)
                    batch_imgs.append(image.img_to_array(img))
                except:
                    pass
            if batch_imgs:
                batch_array = preprocess_input(np.array(batch_imgs))
                batch_features = self.modelo_extrator.predict(batch_array, verbose=0)
                for f in batch_features:
                    normalized = f.flatten() / np.linalg.norm(f.flatten())
                    lista_caracteristicas.append(normalized)
            self.progress_bar.value = (i + len(batch_paths)) / len(caminhos_validos)

        self.vetores_caracteristicas = lista_caracteristicas
        joblib.dump(self.vetores_caracteristicas, "vetores_caracteristicas.joblib")
        joblib.dump(self.caminhos_imagens, "caminhos_imagens.joblib")

        with self.output_area:
            print(f"Treinamento concluído. {len(self.vetores_caracteristicas)} imagens processadas.")
        self.progress_bar.description = 'Concluído!'

    def recomendar(self, caminho_imagem_busca, top_n=5):
        vetor_busca = self._extrair_caracteristicas_imagem(caminho_imagem_busca)
        if vetor_busca is None or not self.vetores_caracteristicas:
            with self.output_area:
                print("Não foi possível gerar recomendações.")
            return
        similarities = cosine_similarity([vetor_busca], np.array(self.vetores_caracteristicas))
        indices = similarities[0].argsort()[::-1]
        self.recomendacoes_atuais = [(self.caminhos_imagens[i], similarities[0][i])
                                     for i in indices if self.caminhos_imagens[i] != caminho_imagem_busca][:top_n]

    def _mostrar_recomendacoes_grid(self):
        if not self.recomendacoes_atuais or self.caminho_busca_atual is None:
            with self.output_area:
                clear_output()
                print("Nenhuma recomendação para exibir.")
            return
        with self.output_area:
            clear_output(wait=True)
            n = len(self.recomendacoes_atuais)
            fig, axes = plt.subplots(1, n+1, figsize=(4*(n+1), 4))
            axes[0].imshow(mpimg.imread(self.caminho_busca_atual))
            axes[0].set_title("Imagem de Busca")
            axes[0].axis("off")
            for i, (caminho, sim) in enumerate(self.recomendacoes_atuais):
                axes[i+1].imshow(mpimg.imread(caminho))
                axes[i+1].set_title(f"{sim:.2f}")
                axes[i+1].axis("off")
            plt.show()

    def _handle_upload(self, change):
        if not self.uploader.value:
            return
        uploaded_file_info = list(self.uploader.value.values())[0]
        filename = uploaded_file_info['metadata']['name']
        content = uploaded_file_info['content']
        self.caminho_busca_atual = os.path.join("uploads", filename)
        with open(self.caminho_busca_atual, 'wb') as f:
            f.write(content)
        self.recomendar(self.caminho_busca_atual)
        self._mostrar_recomendacoes_grid()
        self.uploader.value.clear()
        self.uploader._counter += 1

    def _on_click_treinar(self, b):
        self.treinar()

    def iniciar_interface(self):
        with self.output_area:
            clear_output()
            print("Iniciando interface...")

        if not self.carregar_dados_treinados():
            with self.output_area:
                print("Nenhum catálogo encontrado. Adicione imagens e clique em 'Executar Treinamento'.")
        display(widgets.VBox([
            widgets.HTML("<h3>Sistema de Recomendação por Imagem</h3>"),
            self.uploader,
            self.output_area,
            self.btn_treinar,
            self.progress_bar
        ]))


In [ ]:

# BLOCO 3: EXECUÇÃO E INTERFACE

sistema = SistemaRecomendacao()
sistema.iniciar_interface()
